# NYC High School SAT Data Modeling and Cleaning

#### Data Analysis Portfolio Project
by Giovani Goltara

---

## Executive Summary

This notebook demonstrates the process of cleaning, modeling, and preparing NYC high school SAT data for integration and analysis. Key steps include data quality assessment, normalization, removal of invalid or redundant data, and schema design for downstream analytics.

---

## Data Cleaning Logic

- Initial assessment and cleaning performed using Data Wrangler.
- Normalized column names for consistency.
- Dropped rows with missing or invalid values (e.g., 's' for missing SAT scores).
- Removed duplicate columns and corrected data types.
- Filtered SAT scores to valid range (200-800).
- Retained only the school code as the primary key.
- Dropped unnecessary columns (e.g., school name, contact extension, percent students tested).

## SQL Schema & Integration Strategy

- Used `school_code` as the primary key for unique identification.
- Removed low-quality or redundant columns to simplify the schema.
- Ensured numeric columns have appropriate data types.
- Dropped rows with invalid or missing critical values to maintain data integrity.
- The cleaned table is ready for joining with other datasets on `school_code` for further analysis.
- Downstream integrations should account for the absence of dropped columns.

In [ ]:
# Environment Setup: Import Required Libraries
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Database Connection: Define PostgreSQL Connection String
DATABASE_URL = (
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

In [ ]:
# Data Ingestion: Load SAT Results Dataset
df = pd.read_csv('/Users/giovanigoltara/Documents/webeet/onboarding-webeet/_onboarding_data-1/daily_tasks/day_4/day_4_datasets/sat-results.csv')
df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


In [ ]:
# Data Cleaning: Standardize and Normalize Column Names
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_').str.replace('.', '')
df.head()

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,sat_critical_readng_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


In [ ]:
# Data Cleaning: Remove Duplicates and Handle Missing Values

# Remove duplicate rows
new_df = df.drop_duplicates()

# Remove duplicated column if present
new_df = new_df.drop(columns=['sat_critical_readng_avg_score'], errors='ignore')

# Drop rows with 's' values in key columns
columns_to_check = [
    'num_of_sat_test_takers',
    'sat_critical_reading_avg_score',
    'sat_math_avg_score',
    'sat_writing_avg_score',
    'academic_tier_rating'
    ]

# Keep rows where at least one of the specified columns does not have 's'
new_df = new_df[~(new_df[columns_to_check] == 's').all(axis=1)]

In [ ]:
# Data Integration: Upload Cleaned DataFrame to PostgreSQL Database
new_df.to_sql(
    name='giovani_sat_results',
    con=engine,
    schema='nyc_schools',
    if_exists='replace',
    index=False
    )

416

In [ ]:
# Data Export: Save Cleaned DataFrame to CSV
new_df.to_csv('/Users/giovanigoltara/Documents/webeet/onboarding-webeet/_onboarding_data-1/daily_tasks/day_4/sat-results-cleaned.csv', index=False)